# Taller #48: Detección de Objetos en Tiempo Real con YOLO y Webcam
#### Desarrollado por: David Santiago Cruz Hernández

In [1]:
import cv2
import time
from ultralytics import YOLO

# pip install -r requirements.txt

## Definir el modelo YOLOv12

In [2]:
# Cargar el modelo YOLOv12 preentrenado
model = YOLO('yolo12n.pt')

## Definir las clases a detectar

In [3]:
CLASSES_TO_SHOW = ['person', 'cell phone', 'scissors']

# Mapea los nombres de clase a sus índices
class_names = model.names
class_indices = [k for k, v in class_names.items() if v in CLASSES_TO_SHOW]

## Procesar el video en tiempo real

- ### Dibujar cajas y etiquetas en el frame

In [4]:
def draw_boxes(frame, text, x1, y1, x2, y2):
    """Dibuja una caja delimitadora y un texto en el frame."""
    color = (0, 255, 0)  # Color verde
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

- ### Procesar cada frame y extraer las detecciones

In [5]:
def process_frame(frame, result):
    """Procesa un frame y dibuja las detecciones."""
    boxes = result.boxes  # Obtener las cajas detectadas

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])          # Obtener coordenadas de la caja delimitadora
        conf = box.conf[0]                              # Confianza
        cls_id = int(box.cls[0])                        # ID de clase
        label = class_names[cls_id]                     # Nombre de la clase

        # Filtrar por clases específicas
        if class_names[cls_id] not in CLASSES_TO_SHOW:
            continue

        # Dibujar la caja y el texto en el frame
        text = f"{label} {conf:.2f}"
        draw_boxes(frame, text, x1, y1, x2, y2)

## Captura de video en tiempo real

- ### Calcular FPS

In [6]:
def calculate_fps(prev_frame_time, new_frame_time):
    """Calcula los FPS basados en el tiempo entre frames."""
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    return prev_frame_time, fps

- ### Capturar video desde la webcam y procesar en tiempo real

In [ ]:
# Captura de video desde la webcam
cap = cv2.VideoCapture(0)

# Variables para calcular FPS
prev_frame_time = 0
new_frame_time = 0

print("[INFO] Iniciando cámara... Presiona 'q' para salir.")

while True:
    # Leer un frame de la cámara
    ret, frame = cap.read()
    if not ret:
        print("[ERROR] No se pudo acceder a la cámara.")
        break

    # Actualizar el tiempo del nuevo frame
    new_frame_time = time.time()

    # Realizar inferencia en el frame actual
    results = model.predict(source=frame, stream=True)

    # Procesar resultados
    for result in results:
        process_frame(frame, result)

    # Calcular y mostrar FPS
    prev_frame_time, fps = calculate_fps(prev_frame_time, new_frame_time)
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Mostrar el frame con las detecciones
    cv2.imshow('YOLOv11 - Deteccion en Tiempo Real', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
print("[INFO] Programa terminado.")

[INFO] Iniciando cámara... Presiona 'q' para salir.

0: 480x640 (no detections), 79.9ms
Speed: 2.0ms preprocess, 79.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
